# Extract data from dataset .mat files

In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np  
import glob
import os

mat_path = "matdata/AirShow_HuangBufferBasedAdaptor_Trace_0.mat"
mat = loadmat(mat_path)
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'content_name', 'STRRED', 'N_playback_frames', 'VMAF', 'SSIM', 'buffer_evolution_sec', 'height', 'playback_duration_sec', 'playout_bitrate', 'continuous_zscored_mos', 'scene_cuts_detected', 'PSNR', 'MSSIM', 'per_segment_encoding_QP', 'rebuffer_duration_sec', 'video_duration_sec', 'width', 'per_segment_encoding_height', 'selected_streams', 'distorted_mp4_video', 'adaptation_algorithm', 'rebuffer_number', 'content_spatial_information', 'content_temporal_information', 'frame_rate', 'per_segment_encoding_width', 'cropping_parameters', 'throughput_trace_name', 'content_name_acronym', 'scene_cuts', 'N_rebuffer_frames', 'is_rebuffered_bool', 'throughput_trace_kbps', 'reference_yuv_video', 'N_total_frames', 'retrospective_zscored_mos'])


In [2]:
for key in list(mat.keys())[3:]:
    print(f"{key}: {type(mat[key])}, shape: {mat[key].shape}")

content_name: <class 'numpy.ndarray'>, shape: (1,)
STRRED: <class 'numpy.ndarray'>, shape: (1, 814)
N_playback_frames: <class 'numpy.ndarray'>, shape: (1, 1)
VMAF: <class 'numpy.ndarray'>, shape: (1, 814)
SSIM: <class 'numpy.ndarray'>, shape: (1, 814)
buffer_evolution_sec: <class 'numpy.ndarray'>, shape: (1, 15)
height: <class 'numpy.ndarray'>, shape: (1, 1)
playback_duration_sec: <class 'numpy.ndarray'>, shape: (1, 1)
playout_bitrate: <class 'numpy.ndarray'>, shape: (1, 814)
continuous_zscored_mos: <class 'numpy.ndarray'>, shape: (1, 814)
scene_cuts_detected: <class 'numpy.ndarray'>, shape: (1, 1)
PSNR: <class 'numpy.ndarray'>, shape: (1, 814)
MSSIM: <class 'numpy.ndarray'>, shape: (1, 814)
per_segment_encoding_QP: <class 'numpy.ndarray'>, shape: (1, 15)
rebuffer_duration_sec: <class 'numpy.ndarray'>, shape: (1, 1)
video_duration_sec: <class 'numpy.ndarray'>, shape: (1, 1)
width: <class 'numpy.ndarray'>, shape: (1, 1)
per_segment_encoding_height: <class 'numpy.ndarray'>, shape: (1, 15

In [3]:
mat['throughput_trace_kbps']

array([[1392.1261477 , 1139.87237354,  993.08673267,  682.47833333,
         678.10669111, 1041.3578635 ,  653.67779961,  759.70485437,
         285.93882353,  234.00429752, 1057.52229931, 1158.73246753,
        1505.62597403, 1610.09021956, 1085.84468719, 1015.64752475,
        1368.97662338, 1137.21268583,  779.10857143,  390.9370297 ,
         973.58961039,  715.31428571, 1019.22041217,  983.20998004,
        1083.03555114, 1239.59762376,  735.81029703,  824.05069307,
         539.65194805,  634.51337958,  420.69733333,  717.75584416,
        1047.88503469,  842.32578497, 1641.6       , 1768.11948052,
        1727.41727905, 1479.67462981, 1297.15988083,  884.84544564]])

In [4]:
import numpy as np
import pandas as pd
import glob
import os
from scipy.io import loadmat

def safe_flatten(mat_dict, key):
    """Return flattened array if key exists, else empty np.array()."""
    return np.array(mat_dict[key]).flatten() if key in mat_dict else np.array([])


def extract_per_frame_features(mat_path):
    mat = loadmat(mat_path)

    vmaf = safe_flatten(mat, "VMAF")
    psnr = safe_flatten(mat, "PSNR")
    ssim = safe_flatten(mat, "SSIM")
    strred = safe_flatten(mat, "STRRED")
    bitrate = safe_flatten(mat, "playout_bitrate")
    is_rebuf = safe_flatten(mat, "is_rebuffered_bool")
    cont_mos = safe_flatten(mat, "continuous_zscored_mos")

    # Determine number of valid frames
    n = min(
        len(vmaf),
        len(psnr),
        len(ssim),
        len(bitrate),
        len(cont_mos),
        len(is_rebuf)
    )

    # --- Create frame-level table ---
    df = pd.DataFrame({
        "frame_idx": np.arange(n),
        "vmaf": vmaf[:n],
        "psnr": psnr[:n],
        "ssim": ssim[:n],
        "strred": strred[:n] if len(strred) >= n else np.full(n, np.nan),
        "bitrate": bitrate[:n],
        "is_rebuffered": is_rebuf[:n],
        "continuous_mos": cont_mos[:n]
    })

    # --- Video-level metadata ---
    df["video_name"] = os.path.basename(mat_path).replace(".mat", "")

    df["content_name"] = (
        str(safe_flatten(mat, "content_name")[0])
        if "content_name" in mat else None
    )
    df["adaptation_algorithm"] = (
        str(safe_flatten(mat, "adaptation_algorithm")[0])
        if "adaptation_algorithm" in mat else None
    )
    df["throughput_trace_name"] = (
        str(safe_flatten(mat, "throughput_trace_name")[0])
        if "throughput_trace_name" in mat else None
    )
    df["spatial_info"] = float(safe_flatten(mat, "content_spatial_information")[0]) \
        if "content_spatial_information" in mat else np.nan

    df["temporal_info"] = float(safe_flatten(mat, "content_temporal_information")[0]) \
        if "content_temporal_information" in mat else np.nan

    df["rebuffer_duration_sec"] = float(safe_flatten(mat, "rebuffer_duration_sec")[0]) \
        if "rebuffer_duration_sec" in mat else np.nan

    df["retrospective_mos"] = float(safe_flatten(mat, "retrospective_zscored_mos")[0]) \
        if "retrospective_zscored_mos" in mat else np.nan

    return df


# --- Process all .mat files ---
all_files = glob.glob("matdata/*.mat")

all_dfs = []
for f in all_files:
    print(f"Processing {f} ...")
    all_dfs.append(extract_per_frame_features(f))

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.to_csv("data/per_frame_data.csv", index=False)

print("Saved: data/per_frame_data.csv")

Processing matdata/TearsOfSteelRobot_VMAFViterbiQualityBasedAdaptor_Trace_1.mat ...
Processing matdata/ElFuenteDance_SimpleThroughputBasedAdaptor_Trace_2.mat ...
Processing matdata/Sparks_VMAFViterbiQualityBasedAdaptor_Trace_1.mat ...
Processing matdata/ElFuenteDance_OracleVMAFViterbiQualityBasedAdaptor_Trace_6.mat ...
Processing matdata/CosmosLaundromat_SimpleThroughputBasedAdaptor_Trace_2.mat ...
Processing matdata/Chimera1102347_HuangBufferBasedAdaptor_Trace_0.mat ...
Processing matdata/Chimera1102353_OracleVMAFViterbiQualityBasedAdaptor_Trace_2.mat ...
Processing matdata/Chimera1102353_OracleVMAFViterbiQualityBasedAdaptor_Trace_3.mat ...
Processing matdata/ElFuenteMask_HuangBufferBasedAdaptor_Trace_4.mat ...
Processing matdata/TearsOfSteelStatic_VMAFViterbiQualityBasedAdaptor_Trace_1.mat ...
Processing matdata/AirShow_OracleVMAFViterbiQualityBasedAdaptor_Trace_1.mat ...
Processing matdata/TearsOfSteelStatic_SimpleThroughputBasedAdaptor_Trace_4.mat ...
Processing matdata/MeridianDri